---
Ville Turku | 31 January 2022

---

# Predicting hourly order count

## Introduction

The purpose of this short project is to create a predictive model, which could be used to predict orders for the next hour made in food delivery app. The benefit of predictions generated by this model would be apparent in at least two cases:
* Generating demand estimates for food delivery app restaurant (and retail) partners
* For further use in data pipelines, e.g. as an input for estimating delivery times

For this project, nine predictive models in total will be trained using the Scikit-learn library in Python, using data from food delivery app and Fintraffic, to be compared with one another. Among these nine candidate models, some models will be recommended for further use or evaluation.

## Data

Two distinct datasets will be used for training the predictive models.

The first dataset consists of order data directly from food delivery app, consisting of features such as weather conditions during the hour of the purchase, and naturally the time of the purchase. The data will be aggregated on an hourly level.

In addition to food delivery app's purchase data, complementary open traffic data from Fintraffic will be used as a proxy for activity levels of possible clients, in aggregate. Due to the limited scope of this project, the traffic data will be considered at a very narrow level, which is naturally not ideal - but hopefully enough to evaluate traffic data as a complementary data source. 

To evaluate the usefulness of the traffic data, three different feature selections will be used:
* Only food delivery app's original data
* food delivery app's data, time-related data, combined with traffic data
* food delivery app's original data, and traffic data combined

The idea in the above set-up is to evaluate whether traffic data brings in additional value, or if it is essentially collinear with the original data.

### Libraries

The following libraries were used in this project:

In [24]:
import pandas as pd
import numpy as np
from math import pi

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge

from bokeh.models import ColumnDataSource, FactorRange, LabelSet, Panel, Tabs
from bokeh.plotting import figure, show, output_notebook
from bokeh.transform import jitter

# Data processing

To get started, we start with a quick overview at the data, to aid us in bringing it to a usable form.

In [25]:
data = pd.read_csv("orders_autumn_2020.csv")
data.head()

,TIMESTAMP,ACTUAL_DELIVERY_MINUTES - ESTIMATED_DELIVERY_MINUTES,ITEM_COUNT,USER_LAT,USER_LONG,VENUE_LAT,VENUE_LONG,ESTIMATED_DELIVERY_MINUTES,ACTUAL_DELIVERY_MINUTES,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION
0,2020-08-01 06:07:00.000,-19,1,60.158,24.946,60.160,24.946,29,10,0.0,15.0,3.53644,0.0
1,2020-08-01 06:17:00.000,-7,8,60.163,24.927,60.153,24.910,39,32,0.0,15.0,3.53644,0.0
2,2020-08-01 06:54:00.000,-17,4,60.161,24.937,60.162,24.939,23,6,0.0,15.0,3.53644,0.0
3,2020-08-01 07:09:00.000,-2,3,60.185,24.954,60.190,24.911,28,26,0.0,16.7,3.52267,0.0
4,2020-08-01 07:10:00.000,-1,2,60.182,24.955,60.178,24.949,27,26,0.0,16.7,3.52267,0.0


For this analysis, we will consider values that do not need to be time lagged, for simplicity. Hence, columns such as item count, which would have to be considered for the previous hour, since we cannot know the item count of the ongoing hour in advance. We will also exclude characteristics related to individual orders, such as user latitude. Having established this, we can proceed to picking out the relevant features, and doing some basic data processing and cleaning.

In [26]:
data["TIMESTAMP"] = data["TIMESTAMP"].str.replace(":\d+:\d+.000", ":00:00", regex=True)
data = data.fillna(0)

After that, we can aggregate the data on an hourly basis, counting the number of orders in each hour. Since the order times themselves can be used as a feature (as we are dealing with time series), we pick out hour and day as features. This is in line with the degree of granularity that we are aggregating to. We also cannot use higher level data such as months, for instance, as feature, since we are only dealing with data from August to September.

In [27]:
X = data.groupby("TIMESTAMP")[["CLOUD_COVERAGE", "TEMPERATURE", "WIND_SPEED", "PRECIPITATION"]].agg("min")
X["HOUR"] = X.index.str.slice(11, 13).astype(int)
X.index = pd.to_datetime(X.index)
X["WEEKDAY"] = X.index.dayofweek
X = X.drop("WEEKDAY", axis=1).join(pd.get_dummies(X["WEEKDAY"], drop_first=True, prefix="DAY"))
X.head()

,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION,HOUR,DAY_1,DAY_2,DAY_3,DAY_4,DAY_5,DAY_6
TIMESTAMP,,,,,,,,,,,
2020-08-01 06:00:00,0.0,15.0,3.53644,0.0,6,0,0,0,0,1,0
2020-08-01 07:00:00,0.0,16.7,3.52267,0.0,7,0,0,0,0,1,0
2020-08-01 08:00:00,25.0,17.8,3.35088,0.0,8,0,0,0,0,1,0
2020-08-01 09:00:00,0.0,17.8,3.29756,0.0,9,0,0,0,0,1,0
2020-08-01 10:00:00,0.0,17.8,3.15193,0.0,10,0,0,0,0,1,0


After preprocessing food delivery app's data, we can join the data with our supplemental traffic data. The data is retrieved from https://tie-lam-test.digitraffic.fi/, with the following parameters in the data request form (in Finnish):
* Aineisto: Liikennemäärät
* Raportti: Tuntiliikenneraportti
* Aika
** Alkuaika: 2020-08-01
** Loppuaika: 2020-09-01
* LAM-pisteet: 117 (Tie 1, Munkkiniemi, Helsinki)
* Ajoneuvoluokat: Kaikki
* Suunnat: Kaikki
* Kaistat: Kyllä, kaistat eriteltynä

Hence, we are looking at traffic levels at an hourly level, within the same timeframe as our original data. For the sake of simplicity, we are only evaluating traffic activity in Munkkiniemi, Helsinki, with all vehicle classes, directions and lanes. Of course, this selection of data is not perfect, as it only consists of one data point throughout the entire traffic network. In future, the data could be fitted with a better granularity in tandem with geographical information in purchase data. Additional data points could also be used to get more reliable activity data. For instance, there could be roadwork in the specific location, which would introduce bias in the data - as compared to all possible data points averaged out.

We start by importing the retrieved data and inspecting its form.


In [28]:
traffic_data = pd.read_csv("traffic.csv", sep=";", index_col=["pvm", "kaista"])
traffic_data.head()

pistetunnus         sijainti suunta  suuntaselite jaottelu  \
pvm      kaista                                                               
20200801 1               117  vt1_Munkkiniemi      *           NaN   Kaikki   
         2               117  vt1_Munkkiniemi      *           NaN   Kaikki   
         3               117  vt1_Munkkiniemi      *           NaN   Kaikki   
         4               117  vt1_Munkkiniemi      *           NaN   Kaikki   
         5               117  vt1_Munkkiniemi      *           NaN   Kaikki   

                ajoneuvoluokka  00_01  01_02  02_03  03_04  ...  15_16  16_17  \
pvm      kaista                                             ...                 
20200801 1              kaikki  127.0   99.0   77.0   78.0  ...    549    505   
         2              kaikki   24.0   18.0   15.0   18.0  ...    208    232   
         3              kaikki   46.0   24.0   29.0   13.0  ...    333    346   
         4              kaikki  100.0   68.0   51.0   57.0  ...    449    432   
         5              kaikki    5.0    4.0    4.0    3.0  ...     10     14   

                 17_18  18_19  19_20  20_21  21_22  22_23  23_24  yhteensa  
pvm      kaista                                                             
20200801 1         488    438    440    360    323    254  180.0      7745  
         2         196    172    165    172    102     81   64.0      2827  
         3         332    374    256    192    166    138   89.0      4355  
         4         442    418    380    330    251    198  174.0      6331  
         5          13     14     10     11      6     10    6.0       187  

[5 rows x 31 columns]

From a first glance, we can see that the data comes with many features that are perhaps not so much of interest. It seems that the traffic counts are listed under the columns with numeric headers, denoting hour of the day. Hence, we can proceed by removing the irrelevant features and pivoting the data into a format that is easier to process.

In [29]:
traffic_data = traffic_data.drop(["pistetunnus", "sijainti", "suunta", "suuntaselite", "jaottelu", "ajoneuvoluokka", "yhteensa"], axis=1)
traffic_data = pd.melt(traffic_data, var_name="time", ignore_index=False)
traffic_data = traffic_data.reset_index(level=1)
traffic_data.head()

,kaista,time,value
pvm,,,
20200801,1,00_01,127.0
20200801,2,00_01,24.0
20200801,3,00_01,46.0
20200801,4,00_01,100.0
20200801,5,00_01,5.0


At this step, we still have times as a separate columns. To ensure the right format, we should index the data by the hours, combined with dates, so we can use it with food delivery app's data. To match the data, we also shift back the traffic data by one hour. The reason for this is, that if we were to use this kind of data in production, we certainly would not know in advance the traffic counts for the ongoing hour, but instead the previous one.

In [30]:
traffic_data.index = pd.to_datetime(
  traffic_data.index.astype("str") + " " + traffic_data["time"].str.replace("_\d+", "", regex=True) + ":00") - pd.Timedelta(hours=1)
traffic_data = traffic_data.drop("time", axis=1)
traffic_data

,kaista,value
2020-07-31 23:00:00,1,127.0
2020-07-31 23:00:00,2,24.0
2020-07-31 23:00:00,3,46.0
2020-07-31 23:00:00,4,100.0
2020-07-31 23:00:00,5,5.0
...,...,...
2020-09-30 22:00:00,1,123.0
2020-09-30 22:00:00,2,22.0
2020-09-30 22:00:00,3,30.0
2020-09-30 22:00:00,4,74.0


And finally, we make a column for each distinct lane, with the corresponding traffic count.

In [31]:
traffic_data = traffic_data.pivot(columns="kaista")
traffic_data.columns = "LANE" + traffic_data.columns.droplevel().astype("str")
traffic_data.head()

kaista,LANE1,LANE2,LANE3,LANE4,LANE5
2020-07-31 23:00:00,127.0,24.0,46.0,100.0,5.0
2020-08-01 00:00:00,99.0,18.0,24.0,68.0,4.0
2020-08-01 01:00:00,77.0,15.0,29.0,51.0,4.0
2020-08-01 02:00:00,78.0,18.0,13.0,57.0,3.0
2020-08-01 03:00:00,80.0,17.0,14.0,50.0,2.0


We can then merge the traffic data with the order data.

In [32]:
X = X.merge(traffic_data, left_index=True, right_index=True)
X.head()

,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION,HOUR,DAY_1,DAY_2,DAY_3,DAY_4,DAY_5,DAY_6,LANE1,LANE2,LANE3,LANE4,LANE5
2020-08-01 06:00:00,0.0,15.0,3.53644,0.0,6,0,0,0,0,1,0,161.0,31.0,63.0,109.0,3.0
2020-08-01 07:00:00,0.0,16.7,3.52267,0.0,7,0,0,0,0,1,0,245.0,41.0,89.0,138.0,2.0
2020-08-01 08:00:00,25.0,17.8,3.35088,0.0,8,0,0,0,0,1,0,395.0,109.0,174.0,245.0,8.0
2020-08-01 09:00:00,0.0,17.8,3.29756,0.0,9,0,0,0,0,1,0,499.0,181.0,253.0,386.0,10.0
2020-08-01 10:00:00,0.0,17.8,3.15193,0.0,10,0,0,0,0,1,0,569.0,202.0,293.0,433.0,6.0


As a safety check, we can ensure we have no null values.

In [33]:
X.isnull().any().any()

False

And having defined the features, we can proceed to aggregating the order counts, which is our target variable.

In [34]:
y = data.groupby("TIMESTAMP")["TIMESTAMP"].agg("count")
y.index = pd.to_datetime(y.index)
y.name = "y"
y

TIMESTAMP
2020-08-01 06:00:00     3
2020-08-01 07:00:00     6
2020-08-01 08:00:00    15
2020-08-01 09:00:00    20
2020-08-01 10:00:00    26
                       ..
2020-09-30 16:00:00    42
2020-09-30 17:00:00    26
2020-09-30 18:00:00    19
2020-09-30 19:00:00     8
2020-09-30 20:00:00     1
Name: y, Length: 940, dtype: int64

## Exploratory analysis

Having preprocessed our data, we can then look at relationships in the data, through a brief exploratory data analysis. We start by counting the dimensions of the data.

In [35]:
X.shape

(940, 16)

We have 940 rows and 16 columns in total.

Some descriptive statistics are also in order, which can be used for spotting candidates for outliers.

In [36]:
X.describe().round(2)

,CLOUD_COVERAGE,TEMPERATURE,WIND_SPEED,PRECIPITATION,HOUR,DAY_1,DAY_2,DAY_3,DAY_4,DAY_5,DAY_6,LANE1,LANE2,LANE3,LANE4,LANE5
count,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00,940.00
mean,10.68,16.26,3.48,0.31,12.62,0.14,0.15,0.13,0.15,0.16,0.14,570.37,290.69,401.30,479.66,20.07
std,22.98,4.01,1.53,1.08,4.52,0.35,0.35,0.34,0.35,0.37,0.35,203.90,188.68,183.86,157.26,12.20
min,0.00,0.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,88.00,9.00,37.00,71.00,1.00
25%,0.00,13.90,2.40,0.00,9.00,0.00,0.00,0.00,0.00,0.00,0.00,482.00,193.00,307.00,407.75,11.00
50%,0.00,16.10,3.33,0.00,13.00,0.00,0.00,0.00,0.00,0.00,0.00,558.00,246.00,373.00,488.00,18.00
75%,0.00,18.30,4.37,0.00,16.00,0.00,0.00,0.00,0.00,0.00,0.00,647.00,326.25,489.00,554.00,27.00
max,100.00,26.70,9.86,6.32,22.00,1.00,1.00,1.00,1.00,1.00,1.00,1098.00,912.00,938.00,972.00,62.00


The data seems to be relatively good in terms of veracity, although the weather data is somewhat a cause for concern. Naturally, what matters here is the measurement scale. To get a better look, we can turn to plotting the data. Doing that, we can also look at simple polynomial relationships in the data for preliminary evaluation of feature usefulness.

We start by making a small helper function for evaluating OLS-regressions.

In [37]:
def get_reg(x, y):
  reg = np.polyfit(x, y, 1)
  rng = np.arange(x.min(), x.max(), step = 0.05)
  est = np.array([reg[0] * e + reg[1] for e in rng])
  return {"x": rng, "y": est}

In [38]:
plot_source = ColumnDataSource(X.merge(y, left_index=True, right_index=True))

def make_fig(col):
  p = figure(width=1000, x_axis_label=col, y_axis_label="Hourly purchases")
  p.scatter(jitter(col, 0.1), "y", source=plot_source, alpha=0.4)
  reg = get_reg(X[col].sort_values(), y)
  p.line(reg["x"], reg["y"])
  return Panel(child=p, title=col)

output_notebook()
tabs = Tabs(tabs=[make_fig(c) for c in X.columns])

h = show(tabs, notebook_handle=True)

Loading BokehJS ...

In the interactive plot, we can inspect the features. Generally, the data seems to be in a reasonable order. There are some odd values, such as temperatures of zero. However, at this stage we consider the data as it is due to relatively low number of observations, and limited scope of the project. In future, additional work could be put into ensuring that the data is pristine.

For most independent variables, we can observe a linear relationship with the independent variable, although in some cases, such as with day of the week dummies, the effect is quite weak. It should also be noted that the variables are **not** on a standardized scale.

### Models

We will then proceed to the modeling tasks itself. We have reviously defined three featuresets:
* food delivery app's data only
* food delivery app's data excluding time information together with traffic data
* All of the data together

To evaluate these three featuresets, we pick three models with different purposes:
* Ridge regression, for evaluating a simple linear fit. Ridge regression is chosen because the data most likely suffers from multicollinearity
* A gradient boosting regressor, for a more nuanced fit of the data
* Random forest regressor, to check whether our gradient boosted model is suffering from overfitting

We start by splitting the data into a training and test set, using a relatively large testing set, due to the small amount of the data that we have.

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, test_size=0.5)

We then define models for the "baseline" data, which lacks traffic information.

In [40]:
baseline_X_train = X_train.drop(["LANE1", "LANE2", "LANE3", "LANE4", "LANE5"], axis=1)
baseline_X_test = X_test.drop(["LANE1", "LANE2", "LANE3", "LANE4", "LANE5"], axis=1)

In [41]:
m1 = Ridge(random_state=123)
m1.fit(baseline_X_train, y_train)
m2 = GradientBoostingRegressor(random_state=123)
m2.fit(baseline_X_train, y_train)
m3 = RandomForestRegressor(random_state=123)
m3.fit(baseline_X_train, y_train)

RandomForestRegressor(random_state=123)

Then, we proceed to defining models which include the traffic data, but exclude the original time-related data.

In [42]:
traffic_X_train = X_train.drop(["HOUR", "DAY_1", "DAY_2", "DAY_3", "DAY_4", "DAY_5", "DAY_6"], axis=1)
traffic_X_test = X_test.drop(["HOUR", "DAY_1", "DAY_2", "DAY_3", "DAY_4", "DAY_5", "DAY_6"], axis=1)

In [43]:
m4 = Ridge(random_state=123)
m4.fit(traffic_X_train, y_train)
m5 = GradientBoostingRegressor(random_state=123)
m5.fit(traffic_X_train, y_train)
m6 = RandomForestRegressor(random_state=123)
m6.fit(traffic_X_train, y_train)

RandomForestRegressor(random_state=123)

And finally, we fit models on all features from the original and traffic data combined.

In [44]:
m7 = Ridge(random_state=123)
m7.fit(X_train, y_train)
m8 = DecisionTreeRegressor(random_state=123)
m8.fit(X_train, y_train)
m9 = RandomForestRegressor(random_state=123)
m9.fit(X_train, y_train)

RandomForestRegressor(random_state=123)

## Model evaluation

After having fitted the models, we turn to evaluating the model. As result of this evaluation, additional changes and tuning could be made for the models. However, again due to the limited scope of this project, the results will be served as is, as a possible basis for future work.

We start by plotting the coefficient of determination $r^2$ for each of the nine models.


In [45]:
score_data = [m.score(baseline_X_test, y_test) for m in [m1, m2, m3]] + [m.score(traffic_X_test, y_test) for m in [m4, m5, m6]] + [m.score(X_test, y_test) for m in [m7, m8, m9]]
score_data = [score_data[i * 3] for i in range(3)] + [score_data[i * 3 + 1] for i in range(3)] + [score_data[i * 3 + 2] for i in range(3)]
score_data = pd.Series(score_data)
factors = [
  ("Ridge regression", "Baseline"), ("Ridge regression", "Traffic without time data"), ("Ridge regression", "All features"), 
  ("Gradient boosting", "Baseline"), ("Gradient boosting", "Traffic without time data"), ("Gradient boosting", "All features"),
  ("Random forest", "Baseline"), ("Random forest", "Traffic without time data"), ("Random forest", "All features"),
]

plot_source = ColumnDataSource({
  "x": factors,
  "y": score_data,
  "text": score_data.round(2).astype("str")
})

p = figure(x_range=FactorRange(*factors), y_range=(0, 1), width=1200, y_axis_label=r"\[r^2\]", x_axis_label="Model", title="Model predictive performance")
p.vbar(x="x", top="y", source=plot_source, width=0.5)

labels = LabelSet(x="x", y="y", y_offset=10, text="text", text_align="center", text_font_size={"value": "14px"}, source=plot_source)

p.add_layout(labels)

h = show(p, notebook_handle=True)

Firstly, we can observe from the results that inclusion of the traffic data brings about quite mixed results. When using ridge regression, we have a much better score than the baseline.

The gradient boosted model, on the other hand, seems to be performing increasingly worse as we increase features. This indicates a potential problem with our default hyperparameters, as well a possibility that we are overfitting our model.

Especially the assumption of overfitting is confirmed, to an extent by the dramatic difference between the *all features* gradient boosting and random forest models. In fact, the random forest seems to be slightly benefitting from the inclusion of the traffic data.

Given the current situation, the **random forest** model would be the best candidate for production. However, we could possibly achieve better results by fine-tuning the gradient boosting model.

To get a better view of the impact of individual features in gradient boosting and random forest models, we can plot their importance weights.

In [46]:
importances_data = [np.array(sorted(zip(m.feature_importances_, m.feature_names_in_), key=lambda k: k[0], reverse=True)) for m in [m2, m3, m5, m6, m8, m9]]

def make_fig(mod, title):
  p = figure(width=1200, x_range=importances_data[mod][:,1], y_range=[0, 1], x_axis_label="Feature", y_axis_label="Feature importance, out of total 1")
  p.xaxis.major_label_orientation = pi / 4
  p.vbar(x=importances_data[mod][:,1], top=importances_data[mod][:,0], width=0.5)
  return Panel(child=p, title=title)

tabs = Tabs(tabs=[
  make_fig(0, "Gradient boosting - Baseline"),
  make_fig(1, "Random forest - Baseline"),
  make_fig(2, "Gradient boosting - Traffic without time data"),
  make_fig(3, "Random forest - Traffic without time data"),
  make_fig(4, "Gradient boosting - All features"),
  make_fig(5, "Random forest - All features")
])

h = show(tabs, notebook_handle=True)

From the plots, it is perhaps most apparent that hour of the day is the strongest predictor of order counts. From the weather attributes, wind speed and temperature tend to stand out among the rest.

Between days of the week, and traffic lanes, the lanes tend to dominate in feature importance. This would indicate that we are indeed gaining something from including lanes. However, it should be noted that we might to an extent be merely replacing information that is already there, as indicated by the quite low, however nonzero added predictive power from including the traffic data. Further (statistical) testing could be undertaken to determine whether this is indeed the case.

If this model were to be deployed in a production environment, we would need to foremost consider the fact that our sample only spans two months. Hence, for instance, the temperature feature could lead to somewhat inaccurate predictions. Naturally, we would have to iteratively re-train the model as we would receive more data.

As for the traffic data, the model could again suffer from seasonal effects. However, the traffic data has otherwise been treated such that the model should be (near) production-ready. The reason for that is that we have indeed fitted the model on traffic data from the previous hour in relation to the hour being predicted, and hence the data would be available to pull from Fintrafic's API at the correct time.

## Summary

In this project, a model for predicting the order count for the ongoing hour was developed. The model was developed with Scikit-learn library in Python, using food delivery app's order data and third party traffic data from Fintrafic. Given the current situation of the model, the best candidate for production would be a random forest model, using all of the data combined. Further developments could be made to re-evaluate other models.

As a word of caution, it is very likely that the current model would suffer from seasonal bias, as the model was fitted on data ranging from August to September. However, hour was the strongest predictor in the data, which would perhaps be quite generalizable as a predictor across seasons. Further data collection should be undertaken for continous development of the model.